In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
import os

module_path = os.path.abspath('../') # root directory
if module_path not in sys.path:
    sys.path.append(module_path)

DATA_PATH  = os.path.abspath('../data')

from src.block_collector.config import flr_explorer_url

#  Extract block info

In [47]:
import os

os.path.abspath('')

'/home/horia/Documents/Repos/github-flare-research/block-collector/notebooks'

In [6]:
import requests

# ── Configuration ──────────────────────────────────────────────────────────
BLOCK_NUMBER = 43700858 # The C-chain block you are interested in

# Flarescan API endpoint for a specific block
# Note: This is an unofficial and undocumented endpoint derived from
# inspecting the explorer's network traffic. It may change without notice.
url = f"https://flarescan.com/api/v2/blocks/{BLOCK_NUMBER}"

try:
    # ── 1) Query the Flarescan API ───────────────────────────────────────
    response = requests.get(url)
    response.raise_for_status()  # Check for HTTP errors
    
    block_data = response.json()
    
    # ── 2) Extract the pChainHeight ──────────────────────────────────────
    # The API provides the P-Chain height directly in the response.
    # We look for the 'p_chain_height' key.
    p_chain_height = block_data.get('p_chain_height')

    if p_chain_height is not None:
        print(f"✅ Success!")
        print(f"For C-Chain Block: {BLOCK_NUMBER}")
        print(f"The corresponding P-Chain Height is: {p_chain_height}")
    else:
        print("❌ 'p_chain_height' not found in the Flarescan API response.")

except requests.exceptions.RequestException as e:
    print(f"HTTP Request failed: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

HTTP Request failed: 403 Client Error: Forbidden for url: https://flarescan.com/api/v2/blocks/43700858


In [7]:
response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

## Extract block header

In [14]:
from web3 import Web3
from web3.middleware import ExtraDataToPOAMiddleware
import requests

# ── Configuration ─────────────────────────────────────────────────────────────
C_CHAIN_RPC = "https://flare-api.flare.network/ext/C/rpc"
P_CHAIN_RPC = "https://flare-api.flare.network/ext/bc/P"
BLOCK_NUMBER = 43700858  # ← change to whichever block you need

# ── 1) Set up Web3 + PoA middleware for C-Chain ───────────────────────────────
w3 = Web3(Web3.HTTPProvider(C_CHAIN_RPC))
w3.middleware_onion.inject(ExtraDataToPOAMiddleware(), layer=0)

# ── 2) Fetch the block and pull out the proposer address ─────────────────────
block = w3.eth.get_block(BLOCK_NUMBER)
proposer_addr = block.miner
print(f"Block {BLOCK_NUMBER} proposer reward address: {proposer_addr}")



Block 43700858 proposer reward address: 0x0100000000000000000000000000000000000000


In [15]:
block

AttributeDict({'baseFeePerGas': 25000000000,
 'blockExtraData': '0x',
 'blockGasCost': '0xf4240',
 'difficulty': 1,
 'extDataGasUsed': '0x0',
 'extDataHash': '0x56e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b421',
 'proofOfAuthorityData': HexBytes('0x000000000000000000000000000beafc000000000010a19300000000002065d000000000000a43aa000000000000000000000000000edc0f000000000005568000000000000c8e770000000000000000'),
 'gasLimit': 15000000,
 'gasUsed': 1581427,
 'hash': HexBytes('0x4bc13c9bc6b1b5d0007a29640d9897fca9766028e3ffe32a34ad8f7143b572dd'),
 'logsBloom': HexBytes('0x0002000000004000000000002000000000000000000000000000000400000000000000000000000000000000400000000000000000000000000000000000000000000000000001000040000000000000000000000000000000008000000000080000000000000000000000000000000000000080000000000100000000000000000000000000000000000000000000000000000000000000000400040000000008000000000000000800000020000000000000000000000000000000000000000000020000000000000000000000

In [16]:
# ── 3) Call the P-Chain to get all current validators ────────────────────────
def get_current_validators():
    payload = {
        "jsonrpc": "2.0",
        "method":  "platform.getCurrentValidators",
        "params":  [],
        "id":      1
    }
    resp = requests.post(P_CHAIN_RPC, json=payload)
    resp.raise_for_status()
    return resp.json()["result"]["validators"]

validators = get_current_validators()
print(f"Fetched {len(validators)} validators from P-Chain")

# ── 4) Match miner address to validationRewardOwner.addresses ────────────────
match = next(
    (
        v for v in validators
        if proposer_addr.lower() in
           [addr.lower() for addr in v["validationRewardOwner"]["addresses"]]
    ),
    None
)

# ── 5) Print out the result ─────────────────────────────────────────────────
if match:
    print("\n✅ Found matching validator:")
    print(f"  Node ID:    {match['nodeID']}")
    print(f"  Start Time: {match['startTime']}")
    print(f"  End Time:   {match['endTime']}")
    print("  Reward Addresses:")
    for addr in match["validationRewardOwner"]["addresses"]:
        print(f"   - {addr}")
else:
    print("\n⚠️  No validator’s reward owner matched that address.")


Fetched 112 validators from P-Chain

⚠️  No validator’s reward owner matched that address.


In [21]:
validators[0]

{'txID': 'ZWYU4VBNxrGfGW9ZnroxUxZhBBEqxiwEvp8XbxNDNkwkQRJoS',
 'startTime': '1746014820',
 'endTime': '1751281200',
 'weight': '4295200000000000',
 'nodeID': 'NodeID-CtSfUZd8nvr4RDaQnKRiP9cae15Nrb1bX',
 'stakeAmount': '4295200000000000',
 'rewardOwner': {'locktime': '0',
  'threshold': '1',
  'addresses': ['P-flare1njkr3hlgge6wswrcsle2qpn4ufwagrqatfd7d0']},
 'validationRewardOwner': {'locktime': '0',
  'threshold': '1',
  'addresses': ['P-flare1njkr3hlgge6wswrcsle2qpn4ufwagrqatfd7d0']},
 'delegationRewardOwner': {'locktime': '0',
  'threshold': '1',
  'addresses': ['P-flare1njkr3hlgge6wswrcsle2qpn4ufwagrqatfd7d0']},
 'potentialReward': '0',
 'accruedDelegateeReward': '0',
 'delegationFee': '15.0000',
 'uptime': '99.8106',
 'connected': True,
 'delegatorCount': '11',
 'delegatorWeight': '6339001374000000',
 'delegators': [{'txID': '2bbuF42VJxAarN8EsEpewkeH94g14HU6FAiWdcXTMKJ9xbJ2WW',
   'startTime': '1749812772',
   'endTime': '1751022432',
   'weight': '1827239500000000',
   'nodeID': 

## P-chain explorer

In [22]:
import requests
from avalanche.api import IndexAPI, PlatformVMAPI
from avalanche.datastructures import PlatformChainBlock

# 1) Pull raw P-Chain block bytes via the Index API
INDEX_RPC = "https://flare-api.flare.network/ext/index/P/block"
def get_raw_pchain_block_bytes(height: int) -> bytes:
    payload = {
        "jsonrpc":"2.0",
        "id":1,
        "method":"index.getContainerByIndex",
        "params":{
            "index": height,
            "encoding":"hex"
        }
    }
    r = requests.post(INDEX_RPC, json=payload).json()
    hexdata = r["result"]["bytes"]
    return bytes.fromhex(hexdata[2:])

# 2) Decode the block to grab the proposer NodeID
raw = get_raw_pchain_block_bytes(123456)      # ← your block number here
block = PlatformChainBlock.decode(raw)        # requires avalanche-python-sdk
proposer_node_id = block.get_proposer()

print("Proposer NodeID:", proposer_node_id)

# 3) Map NodeID → rewardAddress via platform.getValidatorsAt
PCHAIN_RPC = "https://flare-api.flare.network/ext/bc/P"
payload = {
    "jsonrpc":"2.0",
    "id":1,
    "method":"platform.getValidatorsAt",
    "params": {"height": str(block.height)}
}
r = requests.post(PCHAIN_RPC, json=payload).json()
for v in r["result"]["validators"]:
    if v["nodeID"] == proposer_node_id:
        print("Proposer rewardAddress:", v["rewardAddress"])
        break


ModuleNotFoundError: No module named 'avalanche'

In [43]:
import requests

# 1. Fetch the raw block bytes
resp = requests.post(
  "https://flare-api.flare.network/ext/index/P/block",
  json={
    "jsonrpc":"2.0","id":1,
    "method":"index.getContainerByIndex",
    "params":{"index":2,"encoding":"hex"}
  }
).json()
raw = bytes.fromhex(resp["result"]["bytes"][2:])
raw

b"\x00\x00\x00\x00\x00\x00\xa7,\xd3\xf6\xf8F@|\xd4|[d\xec\x96'\x82\xedx\x8a%\xd02\xfe\x9bM\xf3y\x93\x94\xb2?p\x00\x00\x00\x00d\x94oV\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x01\xd5\x00\x00\x00\x00\x00\x03sz\xc4\x82\xd3X\x13!\xc5?g\x88-w\xb2\x82\xc2\xc1\x01\x1b[\xf071\xe3\x12!\x91\x81\xe2\xa8\xb8\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x01\x00\x00\x00\x11\x00\x00\x00\x0e\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\xb3F/\xc3\x95h\xbf\x99\xfd4m<\xdc\xfe\x1f\xb9\x00\xf1L\xdc\xf2v\xe3\xc0\xd9_\x81J\xf8Sx$\x00\x00\x00\x07\x00\x00\x00\x00\x00\x00\xc3P\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x01K\xb9\x81L0&RS&\xfeo\xfd\x9b\x8a\xe8\xd9\xaa\xf6y\xff\x00\x00\x00\x00\x00\x00\x00VPlatformVM utility method buildImportTx to import AVAX to the P-Chain from the C-Chainw\xd3\x07M\xc5\x10\xf4;\t\xac[\xe7~\xde\xe2v\xef;U\xf0\t}PHF\xaa\x8e\xeca?\xc6%\x00\x00\x00\x01\xe

In [52]:
import importlib.metadata
dist = importlib.metadata.distribution("ava-labs-avalanche-protocolbuffers-python")
for file in dist.files:
    print(file)


aliasreader/aliasreader_pb2.py
appsender/appsender_pb2.py
ava_labs_avalanche_protocolbuffers_python-31.1.0.1.dev+39f35551d76c.dist-info/INSTALLER
ava_labs_avalanche_protocolbuffers_python-31.1.0.1.dev+39f35551d76c.dist-info/METADATA
ava_labs_avalanche_protocolbuffers_python-31.1.0.1.dev+39f35551d76c.dist-info/RECORD
ava_labs_avalanche_protocolbuffers_python-31.1.0.1.dev+39f35551d76c.dist-info/REQUESTED
ava_labs_avalanche_protocolbuffers_python-31.1.0.1.dev+39f35551d76c.dist-info/WHEEL
http/http_pb2.py
http/responsewriter/responsewriter_pb2.py
io/reader/reader_pb2.py
io/writer/writer_pb2.py
messenger/messenger_pb2.py
net/conn/conn_pb2.py
p2p/p2p_pb2.py
platformvm/platformvm_pb2.py
rpcdb/rpcdb_pb2.py
sdk/sdk_pb2.py
sharedmemory/sharedmemory_pb2.py
signer/signer_pb2.py
sync/sync_pb2.py
validatorstate/validator_state_pb2.py
vm/runtime/runtime_pb2.py
vm/vm_pb2.py
warp/message_pb2.py


In [66]:
import p2p.p2p_pb2 as p

# List all top-level protobuf message names
print(list(p.DESCRIPTOR.message_types_by_name.keys()))


['Message', 'Ping', 'Pong', 'Handshake', 'Client', 'BloomFilter', 'ClaimedIpPort', 'GetPeerList', 'PeerList', 'GetStateSummaryFrontier', 'StateSummaryFrontier', 'GetAcceptedStateSummary', 'AcceptedStateSummary', 'GetAcceptedFrontier', 'AcceptedFrontier', 'GetAccepted', 'Accepted', 'GetAncestors', 'Ancestors', 'Get', 'Put', 'PushQuery', 'PullQuery', 'Chits', 'AppRequest', 'AppResponse', 'AppError', 'AppGossip']


In [1]:
import requests

P_CHAIN_RPC = "https://flare-api.flare.network/ext/bc/P"
height = 2  # your P-Chain height

# 1) Fetch the block as JSON (includes proposer)
resp = requests.post(
    P_CHAIN_RPC,
    json={
        "jsonrpc": "2.0",
        "id": 1,
        "method": "platform.getBlockByHeight",
        "params": {
            "height": height,
            "encoding": "json"
        }
    },
).json()
block = resp["result"]["block"]
proposer_node_id = block["header"]["proposer"]
print("Proposer NodeID:", proposer_node_id)


KeyError: 'header'

In [2]:
import requests
import json
from web3 import Web3

# Configuration for your LOCAL node
C_CHAIN_RPC = "http://127.0.0.1:9650/ext/bc/C/rpc"

def prep_connection(rpc_node):
    w3 = Web3(Web3.HTTPProvider(rpc_node))
    if not w3.is_connected():
        print("❌ Failed to connect to local node.")
        exit()
    else:
        print("Connected to local node")
    return w3

def get_latest_block_num(rpc_node: str):
    w3 = prep_connection(rpc_node)
    latest_block_num = w3.eth.block_number
    if latest_block_num < 1:
        print("Node is running but no blocks produced yet. Please wait a few seconds and run again.")

    print(f"Node is connected. Latest C-Chain block is: {latest_block_num}")
    exit()
    return latest_block_num

get_latest_block_num(C_CHAIN_RPC)

Connected to local node
Node is running but no blocks produced yet. Please wait a few seconds and run again.
Node is connected. Latest C-Chain block is: 0


0

In [2]:
from web3 import Web3

# --- Configuration for your LOCAL node ---
C_CHAIN_RPC = "http://127.0.0.1:9650/ext/bc/C/rpc"
w3 = Web3(Web3.HTTPProvider(C_CHAIN_RPC))

# --- The address that SHOULD be funded in the default genesis ---
funded_address = "0x8db97C7cEcE249c2b98bDC0226Cc4C2A57BF52FC"

print(f"Attempting to connect to node at {C_CHAIN_RPC}...")

try:
    if not w3.is_connected():
        print("❌ Failed to connect. Is the node running?")
        exit()
    
    print("✅ Node is connected.")
    
    chain_id = w3.eth.chain_id
    print(f"   EVM Chain ID: {chain_id}")
    
    print(f"   Querying balance for genesis account: {funded_address}")
    
    # --- Check the balance of the pre-funded account ---
    balance_wei = w3.eth.get_balance(funded_address)
    balance_eth = w3.from_wei(balance_wei, 'ether')
    
    print(f"\n✅ Success! The balance is: {balance_eth} ETH")
    
    if balance_eth > 0:
        print("\n   This confirms the genesis account IS funded.")
    else:
        print("\n   This confirms the genesis account IS NOT funded.")

except Exception as e:
    print(f"\n❌ An error occurred during check: {e}")

Attempting to connect to node at http://127.0.0.1:9650/ext/bc/C/rpc...
✅ Node is connected.
   EVM Chain ID: 162
   Querying balance for genesis account: 0x8db97C7cEcE249c2b98bDC0226Cc4C2A57BF52FC

✅ Success! The balance is: 1000000000000000 ETH

   This confirms the genesis account IS funded.


In [18]:
from web3 import Web3

# --- Configuration for your LOCAL node ---
C_CHAIN_RPC = "http://127.0.0.1:9650/ext/bc/C/rpc"
w3 = Web3(Web3.HTTPProvider(C_CHAIN_RPC))

# --- The correct private key for the funded genesis account ---
funded_private_key = "0x56289e99c94b6912bfc12adc093c9b51124f0dc54ac7a766b2bc5ccf558d8027"
funded_account = w3.eth.account.from_key(funded_private_key)

print(f"✅ Node is connected (Chain ID: {w3.eth.chain_id})")
print(f"   Using pre-funded account: {funded_account.address}")

try:
    # 1. Build the transaction dictionary
    tx_dict = {
        "from": funded_account.address,
        "to": "0x000000000000000000000000000000000000dEaD",
        "value": w3.to_wei(0.001, 'ether'),
        "gas": 21000,
        "gasPrice": w3.eth.gas_price,
        "nonce": w3.eth.get_transaction_count(funded_account.address),
        "chainId": w3.eth.chain_id
    }

    # 2. Sign the transaction locally with our private key
    signed_tx = w3.eth.account.sign_transaction(tx_dict, funded_private_key)
    print("\nTransaction signed locally.")

    # 3. Send the RAW, signed transaction to the node
    #    FIX: Changed .rawTransaction to .raw_transaction
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    print(f"Raw transaction sent! Waiting for receipt... Hash: {tx_hash.hex()}")
    
    # 4. Wait for the transaction to be mined
    receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    print("\n✅✅✅ Success! Transaction was mined!")
    print(f"      A new block has been produced at number: {receipt.blockNumber}")
    print("      You can now inspect it.")

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Node is connected (Chain ID: 162)
   Using pre-funded account: 0x8db97C7cEcE249c2b98bDC0226Cc4C2A57BF52FC

Transaction signed locally.
Raw transaction sent! Waiting for receipt... Hash: 3bfb04146064b4711c8c5ebd9991231a69a5bd79f9237a8a7a9fce718bf9d624

✅✅✅ Success! Transaction was mined!
      A new block has been produced at number: 3
      You can now inspect it.


In [22]:
import requests
import json
from web3 import Web3

C_CHAIN_RPC = "http://127.0.0.1:9650/ext/bc/C/rpc"

try:
    w3 = Web3(Web3.HTTPProvider(C_CHAIN_RPC))
    print(f"✅ Node is connected.")

    latest_block_num = w3.eth.block_number
    print(f"   Latest C-Chain block is: {latest_block_num}")

    if latest_block_num == 0:
        print("❌ No new blocks to inspect yet.")
        exit()

    target_block = w3.eth.get_block(latest_block_num)
    block_hash = target_block['hash'].hex()
    print(f"   Inspecting block {latest_block_num} with hash: {block_hash}")

    payload = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "avalanche.getBlock",
        "params": {"blockID": block_hash, "encoding": "json"}
    }
    headers = {'Content-Type': 'application/json'}

    response = requests.post(C_CHAIN_RPC, headers=headers, data=json.dumps(payload))
    response.raise_for_status()

    full_block_data = response.json().get('result')

    if full_block_data:
        print("\n✅✅✅ Success! Retrieved the full proposervm.Block wrapper.")
        print("-------------------------------------------------------")
        print(json.dumps(full_block_data, indent=2))
        print("-------------------------------------------------------")

        p_chain_height = full_block_data.get('pChainHeight')
        # The proposer field may be named differently, e.g., 'proposer' or 'cert'
        proposer_info = full_block_data.get('proposer') or full_block_data.get('cert')
        
        print(f"\nP-Chain Height from wrapper: {p_chain_height}")
        print(f"Proposer Info from wrapper: {proposer_info}")
    else:
        print("\n❌ Received a null result from the node.")

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Node is connected.
   Latest C-Chain block is: 1

❌ An error occurred: The field extraData is 86 bytes, but should be 32. It is quite likely that you are connected to a POA chain. Refer to http://web3py.readthedocs.io/en/stable/middleware.html#proof-of-authority for more details. The full extraData is: HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000')


In [27]:
import requests, json, binascii
from pathlib import Path

INDEX = "http://127.0.0.1:9650/ext/index/C/block"

# last accepted
resp = requests.post(INDEX, json={
    "jsonrpc": "2.0", "id": 1,
    "method":  "index.getLastAccepted",
    "params":  {"encoding": "hex"}
}).json()["result"]

raw_hex = resp["bytes"]
Path("latest_proposer_block.hex").write_text(raw_hex[2:] if raw_hex.startswith("0x") else raw_hex)
print(f"saved {resp['id']} ({resp['index']}) to latest_proposer_block.hex")

# decode later with Go / avalanchejs / CLI as shown above


saved 2QUfspoSViNUc8VnkvNFnWoFFFA2eDH8b7i4CCUUSu8Xj9yppB (0) to latest_proposer_block.hex


In [28]:
resp

{'id': '2QUfspoSViNUc8VnkvNFnWoFFFA2eDH8b7i4CCUUSu8Xj9yppB',
 'bytes': '0xf902edf90276a0ae950c7fff81f412efc9a2e35146d1bd81acae17e90833759bf18f6d55869b55a01dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347940100000000000000000000000000000000000000a0fc14baec4ecb1d3e8d6e5ace03f287b91e19f8d4e8cdd7b4015a56b08c856fdca0f4170f1a4c243c256905bab32eee4ca269074dffeb7eed7bad943ddfa371f85ba042a74a21725b8bd586fe13bb4df846035e15a3914e71459fafda707faee97ef6b9010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000400000000000000000000000000000001080000000000000000000000000000000000000000400000000000000000000000000000000000000000000000000000000000000000000000000000000000000200000000000080000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010183e4e1c0825208846863

In [32]:
import json, rlp, sys, pathlib

hex_str = '0xf902edf90276a0ae950c7fff81f412efc9a2e35146d1bd81acae17e90833759bf18f6d55869b55a01dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347940100000000000000000000000000000000000000a0fc14baec4ecb1d3e8d6e5ace03f287b91e19f8d4e8cdd7b4015a56b08c856fdca0f4170f1a4c243c256905bab32eee4ca269074dffeb7eed7bad943ddfa371f85ba042a74a21725b8bd586fe13bb4df846035e15a3914e71459fafda707faee97ef6b9010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000400000000000000000000000000000001080000000000000000000000000000000000000000400000000000000000000000000000000000000000000000000000000000000000000000000000000000000200000000000080000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000010183e4e1c0825208846863f9e4b8560000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000a00000000000000000000000000000000000000000000000000000000000000000880000000000000000a056e81f171bcc55a6ff8345e692c0f86e5b48e01b996cadc001622fb5e363b4218534630b8a008080f86ff86d808534630b8a0082520894000000000000000000000000000000000000dead87038d7ea4c6800080820168a0bd9f30f9f85274dfaf479627ddf8435ca05bc967d8d7fbfa66e07d1f7cfb1f87a0769af7999355e4570c3878f7a2f7907860e26c66e1bf8218cd9ac2cdcfe4d2ffc080800c487d78'
if hex_str.startswith("0x"):
    hex_str = hex_str[2:]

raw = bytes.fromhex(hex_str)
decoded = rlp.decode(raw, strict=False)          # → [header, txs, uncles]


out = {
    name: ("0x"+val.hex() if name not in {
        "difficulty","number","gasLimit","gasUsed","timestamp","baseFeePerGas"}
        else b2i(val))
    for name, val in zip(fields, header)
}

decoded


[[b'\xae\x95\x0c\x7f\xff\x81\xf4\x12\xef\xc9\xa2\xe3QF\xd1\xbd\x81\xac\xae\x17\xe9\x083u\x9b\xf1\x8fmU\x86\x9bU',
  b'\x1d\xccM\xe8\xde\xc7]z\xab\x85\xb5g\xb6\xcc\xd4\x1a\xd3\x12E\x1b\x94\x8at\x13\xf0\xa1B\xfd@\xd4\x93G',
  b'\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
  b'\xfc\x14\xba\xecN\xcb\x1d>\x8dnZ\xce\x03\xf2\x87\xb9\x1e\x19\xf8\xd4\xe8\xcd\xd7\xb4\x01ZV\xb0\x8c\x85o\xdc',
  b'\xf4\x17\x0f\x1aL$<%i\x05\xba\xb3.\xeeL\xa2i\x07M\xff\xeb~\xed{\xad\x94=\xdf\xa3q\xf8[',
  b'B\xa7J!r[\x8b\xd5\x86\xfe\x13\xbbM\xf8F\x03^\x15\xa3\x91NqE\x9f\xaf\xdap\x7f\xae\xe9~\xf6',
  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10\x80\x00\x00\x00\x00\x00\x0

In [45]:
def b2i(b): return int.from_bytes(b, "big") if b else 0

fields = [
    "parentHash", "ommersHash", "beneficiary", "stateRoot", "transactionsRoot",
    "receiptsRoot", "logsBloom", "difficulty", "number", "gasLimit",
    "gasUsed", "timestamp", "extraData", "mixHash", "nonce",
]

for j in decoded[1][0]:
    print(b2i(j))

0
225000000000
21000
57005
1000000000000000
0
360
85768395088539845564665048521297002376692954095306703349142508973742007984007
53646719452537043133514035925212437774493009094068298707569403365806664110847


In [37]:
len(decoded[0]

19

In [43]:
decoded[1]

[[b'',
  b'4c\x0b\x8a\x00',
  b'R\x08',
  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xde\xad',
  b'\x03\x8d~\xa4\xc6\x80\x00',
  b'',
  b'\x01h',
  b"\xbd\x9f0\xf9\xf8Rt\xdf\xafG\x96'\xdd\xf8C\\\xa0[\xc9g\xd8\xd7\xfb\xfaf\xe0}\x1f|\xfb\x1f\x87",
  b'v\x9a\xf7\x99\x93U\xe4W\x0c8x\xf7\xa2\xf7\x90x`\xe2lf\xe1\xbf\x82\x18\xcd\x9a\xc2\xcd\xcf\xe4\xd2\xff']]